In [1]:
from sail.core import VMSetup, registersafeobj,login,connect,pushsafeobj,querydata,get_fns
from sail.data import RemoteDataFrame, Utils
from sail.algo import fdxgb
import time
start_time = time.time()

In [2]:
import json, requests
soPath = "/Development/Milestone3/EndPointTools/SafeObjectEditor/KPMG/safeobjects"
contractdict = [{"DigitalContractGuid":"{3D99F9E7-6959-44AF-B561-4DFF33F71022}"}, {"DigitalContractGuid":"{391BDC04-E76A-4F88-8256-C56B653F82C9}"}]
backendIP = "52.150.26.47"

registersafeobj(soPath)
print("[P]safe objects registered")

[P]safe objects registered


In [3]:
eosb = login("r@r.r", "SailPassword@123", 6200, backendIP)
print("[P]login success")
print(eosb)

[P]login success
5iEQAhtloSOFogbuEsLeHGsSm4ahEdllcvGMBbA69ACxuxoRamb1ApBRQNeq6wKEAw8Ftw0DAAAyzqZbd1pN3oDP0dn0EYtPDQX5DR8OwrYegI3EpixUR4uiNgMspBV7bCdY1lgXyZKhGsY9QwXwLZKX7vq0n9CBd9ywrVLyB0e97xpmsy+Uzidg1W3AHmI4e51uRpoqE4Tu/dGpfE2+ki/BTuQMbgDU9zdRu+6UZKTd9NVG9lBNmbFBaNJvFwwoBcchDtw8sxhRNdLEdO46utcVfB2MKIAw0jm755TKKTiUNEQ6cxXpNJZj9YNlKdX7pQSJhON8+B/K5di3MvTjbL2Qa8SpELR0yydAO4JPVqb4dg84E6HcJ01RwiR6IwF5vSHUPc30H8Dio9jU5eVlaJjHhzDFET3U0bT/xue5kbXStJ4PKFxHmcEw5qNK0hJK/fGYBAgY2N/azpEO/kUx4kBkhRspxBrvoYEgls7CbY3/pT9+S3cN820rWT2CAUti4pKVipc/+U7KaT59VL3iOu4NHy5CR/Vz4jgqdycTsRTS9X4y2TYLq6Ablo207SPRw6Ho5e/fE9fm5iRJPu1NjBx+x3G1thA8ATFsXDHrOsGL7MkKTuw/+ULQkA2g+GS1teLkzpfMi9Tri0kFge6u2KOYF7i5UI7lCfqusiK1UsJGbPpIm5VUrRO70TnDxDlIRIWn1mm3yLvArUiik2UZ74kOPSBHmsizTKwZGE4GDWxVXLZOO3vInhUlFN+JFM6g5UXrCGCBGGvyn0yzoK1QAHksEcOWaMDmAVARGPhlVETsgaMR/0qXerQytTDFQjtpeD62yVYZTtHIBqma1jmfX3+MwzD3Dp59+iWmLtf/ENh9jxn+yRRxTENUrHlzmyRza5ConO4+Ibc6vLULg+avrJT3FHSR83MsSC6VtNxbEWLXDCxsllRAPqxmeJigSCv9o/IsNYES6JJ2gp/ekAwCR3l

In [4]:
url = "https://"+backendIP+":6200/SAIL/VirtualMachineManager/GetRunningVMsIpAdresses?Eosb="+eosb

payloads = []
for item in contractdict:
    temp = json.dumps(item)
    payloads.append(temp)

headers = {
    'Content-Type': 'application/json'
}

ips = []
for payload in payloads:
    print(payload)
    response = requests.request("GET", url, headers=headers, data=payload, verify=False)
    response = response.json()
    print(response)
    for key in response['VirtualMachines']:
        ips.append(response['VirtualMachines'][key])
        print(ips)

{"DigitalContractGuid": "{3D99F9E7-6959-44AF-B561-4DFF33F71022}"}


/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '52.150.26.47'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'Status': 200.0, 'VirtualMachines': {'ECCCE3A1785C4594A2F626826E9FD97E': '20.102.60.41'}}
['20.102.60.41']
{"DigitalContractGuid": "{391BDC04-E76A-4F88-8256-C56B653F82C9}"}
{'Status': 200.0, 'VirtualMachines': {'48DAD1EFC8A044F89BBB374988602836': '20.120.95.58'}}
['20.102.60.41', '20.120.95.58']


/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '52.150.26.47'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
vmids = []
for ip in ips:
    vmid = connect(ip, 3500)
    vmids.append(vmid)
print("[P]virtual machine connection to ip: {0} success".format(ip), flush=True)
    
fns = get_fns()
for vm in vmids:
    for key in fns:
        pushsafeobj(vm, fns[key])
print("[P]safe object pushed to virtual machines" , flush=True)

table = []
for vm in vmids:
    tableid = querydata(vm)
    table.append(tableid)
print("[P]obtain table ids", flush=True)

[P]virtual machine connection to ip: 20.120.95.58 success
[P]safe object pushed to virtual machines
[P]obtain table ids


In [6]:
table

[{'imbalanced credit card fraud 1': '768D80FEC210E8468B4F027AABCAFB42'},
 {'imbalanced credit card fraud 2': 'F5D68A24D86DFC45BA3CC79CED4EFA0D'}]

In [7]:
#read in datasets

mid_time = time.time()

u1 = Utils(vmids[0], fns)
u2 = Utils(vmids[1], fns)
df1 = u1.read_csv(table[0]['imbalanced credit card fraud 1'])
df2 = u2.read_csv(table[1]['imbalanced credit card fraud 2'])
dfs = [df1, df2]

print(time.time() - mid_time)

16.017621755599976


In [8]:
#create remote dataframes so we can drop the label
n=2
data1 = RemoteDataFrame(vmids[0], df1, fns)
data2 = RemoteDataFrame(vmids[1], df2, fns)
data = [data1, data2]

In [9]:
#print columns of the remote dataframe, use this to set our fdgxb feature number
cols = []
cols.append(data1.columns)
cols.append(data2.columns)

dim = len(cols[0])-2
fdxgbmod = fdxgb(vmids, params={'max_depth': 8 }, feature_num = dim)
print(cols[0])

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [10]:
#drop label from X data, select only label for y data

y=[]
X=[]
n=2
for i in range(n):
    y.append(data[i]['Class'])
    X.append(data[i].drop('Class', axis = 1))

In [11]:
#Check value counts for y
print(y[0].value_counts())
print(y[1].value_counts())

0    142133
1       269
Name: Class, dtype: int64
0    142180
1       223
Name: Class, dtype: int64


In [12]:
#change y from remote series to the corresponding data-ids
y_new = []
for item in y:
    y_new.append(item.data_id)
y=y_new

['{CA497E4C-0329-47D8-9722-E69127019061}.048E16B349F8450694B1B30E0E49E6CE',
 '{23DE9E39-3B57-44F4-8581-CE989F29A8F6}.048E16B349F8450694B1B30E0E49E6CE']

In [13]:
#check that x is actually the correct shape
test1 = RemoteDataFrame(vmids[0], X[0], fns)
test1.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')

In [14]:
#ensure y is a data id and not a remote series
y

['{CA497E4C-0329-47D8-9722-E69127019061}.048E16B349F8450694B1B30E0E49E6CE',
 '{23DE9E39-3B57-44F4-8581-CE989F29A8F6}.048E16B349F8450694B1B30E0E49E6CE']

In [15]:
#ensure X is a data id and not a remote dataframe
X

['{8D1D16C2-3C81-45DE-A241-8BCCC7CABF70}.2AE27D27AE4C4B6BAD3136BAC75B1667',
 '{633A6D9A-EE06-4A6D-A431-452CBD7D3297}.2AE27D27AE4C4B6BAD3136BAC75B1667']

In [16]:
#run training

mid_time = time.time()

model = fdxgbmod.fit(X,y)

print(time.time() - mid_time)

job: {BE5AF29A-42CD-4574-A460-A0C875CE6132} is failed
job: {7BD030DF-6E41-4239-A9A8-F5296BB48B7E} is failed


TypeError: 'NoneType' object is not subscriptable

In [ ]:
mid_time = time.time()
results =  fdxgbmod.aucpr_score(model, X, y)
print(time.time() - mid_time)

In [ ]:
results

In [ ]:
precision = results[0][0]
recall = results[0][1]
import numpy as np
#precision = np.append(precision, [1])
#print(precision)
#print(recall)
from sklearn.metrics import PrecisionRecallDisplay
import matplotlib.pyplot as plt
disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()

In [ ]:
mid_time = time.time()

fdxgbmod.model = model
fdxgbmod.shap(X)

print(time.time() - mid_time)